In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import pickle
data = pd.read_csv('diseasedata.csv')

data.columns = data.columns.str.strip()

data.rename(columns={
    'dischromic _patches': 'dischromic_patches',
    'spotting_ urination': 'spotting_urination',
    'fluid_overload.1': 'fluid_overload'
}, inplace=True)

if 'Unnamed: 133' in data.columns:
    data = data.drop(columns=['Unnamed: 133'])

le = LabelEncoder()
data['prognosis'] = le.fit_transform(data['prognosis'])

X = data.drop('prognosis', axis=1)
y = data['prognosis']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Save the model and label encoder
with open('disease_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

print("Model training complete. Accuracy on test set:", model.score(X_test, y_test))

# # K-Fold Cross-Validation
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# dt_cv_scores = cross_val_score(dt_model, X, y, cv=kf)
# print(f"Decision Tree Cross-Validation Scores: {dt_cv_scores}")
# print(f"Decision Tree Cross-Validation Mean Score: {dt_cv_scores.mean()}")

# # Train a Random Forest model for feature importance analysis
# rf_model = RandomForestClassifier()
# rf_model.fit(X_train, y_train)

# # Feature importance analysis
# feature_importances = pd.DataFrame(rf_model.feature_importances_, index=X.columns, columns=['importance']).sort_values('importance', ascending=False)
# print("Feature Importances:")
# print(feature_importances)

# # Introduce noise to the dataset
# def add_noise(data, noise_level=0.1):
#     noisy_data = data.copy()
#     for col in noisy_data.columns:
#         if pd.api.types.is_numeric_dtype(noisy_data[col]):
#             # Introduce noise by flipping some of the binary features
#             mask = np.random.rand(len(noisy_data)) < noise_level
#             noisy_data.loc[mask, col] = 1 - noisy_data.loc[mask, col]
#     return noisy_data

# X_train_noisy = add_noise(X_train, noise_level=0.1)
# X_test_noisy = add_noise(X_test, noise_level=0.1)

# # Train a Logistic Regression model on noisy data
# from sklearn.linear_model import LogisticRegression

# lr_model = LogisticRegression(max_iter=10000)
# lr_model.fit(X_train_noisy, y_train)

# # Evaluate the Logistic Regression model on noisy test data
# lr_accuracy = lr_model.score(X_test_noisy, y_test)
# print(f"Logistic Regression Accuracy with Noise: {lr_accuracy}")

# # Evaluate the Decision Tree model on the original test data
# dt_accuracy = dt_model.score(X_test, y_test)
# print(f"Decision Tree Accuracy on original data: {dt_accuracy}")

# # Evaluate the Random Forest model on the original test data
# rf_accuracy = rf_model.score(X_test, y_test)
# print(f"Random Forest Accuracy on original data: {rf_accuracy}")


Model training complete. Accuracy on test set: 1.0


In [3]:
data.columns.tolist()

['itching',
 'skin_rash',
 'nodal_skin_eruptions',
 'continuous_sneezing',
 'shivering',
 'chills',
 'joint_pain',
 'stomach_pain',
 'acidity',
 'ulcers_on_tongue',
 'muscle_wasting',
 'vomiting',
 'burning_micturition',
 'spotting_urination',
 'fatigue',
 'weight_gain',
 'anxiety',
 'cold_hands_and_feets',
 'mood_swings',
 'weight_loss',
 'restlessness',
 'lethargy',
 'patches_in_throat',
 'irregular_sugar_level',
 'cough',
 'high_fever',
 'sunken_eyes',
 'breathlessness',
 'sweating',
 'dehydration',
 'indigestion',
 'headache',
 'yellowish_skin',
 'dark_urine',
 'nausea',
 'loss_of_appetite',
 'pain_behind_the_eyes',
 'back_pain',
 'constipation',
 'abdominal_pain',
 'diarrhoea',
 'mild_fever',
 'yellow_urine',
 'yellowing_of_eyes',
 'acute_liver_failure',
 'fluid_overload',
 'swelling_of_stomach',
 'swelled_lymph_nodes',
 'malaise',
 'blurred_and_distorted_vision',
 'phlegm',
 'throat_irritation',
 'redness_of_eyes',
 'sinus_pressure',
 'runny_nose',
 'congestion',
 'chest_pain',
 

In [4]:
app_code = """
import streamlit as st
import pandas as pd
import pickle

# Load the trained model and label encoder
with open('disease_model.pkl', 'rb') as f:
    model = pickle.load(f)
with open('label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)

st.title('Disease Prognosis Prediction System')

# Load and clean the feature names from the dataset
data = pd.read_csv('diseasedata.csv')
data.columns = data.columns.str.strip()
data.rename(columns={
    'dischromic _patches': 'dischromic_patches',
    'spotting_ urination': 'spotting_urination',
    'fluid_overload.1': 'fluid_overload'
}, inplace=True)
if 'Unnamed: 133' in data.columns:
    data = data.drop(columns=['Unnamed: 133'])

feature_names = data.columns[:-1].tolist()

# Translation dictionary for symptoms
translations = {
    'itching': 'រមាស់',
    'skin_rash': 'រលាកស្បែក',
    'nodal_skin_eruptions': 'រោគសើរស្បែក',
    'continuous_sneezing': 'កណ្តស់ជាប់ៗគ្នា',
    'shivering': 'ញ័រញ៉ាក់',
    'chills': 'រងារ',
    'joint_pain': 'ឈឺសន្លាក់',
    'stomach_pain': 'ឈឺពោះ',
    'acidity': 'ឡើងជាត់អាស៊ីត',
    'ulcers_on_tongue': 'ស្នាមជាំលើអណ្តាត',
    'muscle_wasting': 'របួសដាច់សាច់ដុំ',
    'vomiting': 'ហើម',
    'burning_micturition': 'ឈឺនៅពេលបញ្ចេញទឹកនោម',
    'spotting_urination': 'នោមញឹក',
    'fatigue': 'អស់កំលាំង',
    'weight_gain': 'ឡើងទម្ងន់',
    'anxiety': 'ខ្វល់ខ្វាយច្រើន',
    'cold_hands_and_feets': 'ដៃនិងជើងត្រជាក់',
    'mood_swings': 'ប្តូរអារម្មណ៍គិតច្រើន',
    'weight_loss': 'ស្រកគីឡួ',
    'restlessness': 'ភាពមិនសុខស្រួលក្នុងខ្លួន',
    'lethargy': 'ភាពអស់កំលាំង',
    'patches_in_throat': 'បន្ទាត់ក្នុងបំពង់ក',
    'irregular_sugar_level': 'កម្រិតស្ករមិនទៀងទាត់',
    'cough': 'ក្អក',
    'high_fever': 'គ្រុនខ្លាំង',
    'sunken_eyes': 'ភ្នែកចុះច្រាស',
    'breathlessness': 'ភាពមិនអាចដកដង្ហើមបាន',
    'sweating': 'ការបែកញើស',
    'dehydration': 'ភាពស្រេកទឹក',
    'indigestion': 'ការមិនទទួលទានអាហារបានល្អ',
    'headache': 'ឈឺក្បាល',
    'yellowish_skin': 'ស្បែកលឿង',
    'dark_urine': 'ទឹកនោមងងឹត',
    'nausea': 'ការមិនស្រួលក្នុងពោះ',
    'loss_of_appetite': 'បាត់បង់ចំណង់អាហារ',
    'pain_behind_the_eyes': 'ឈឺពីក្រោយភ្នែក',
    'back_pain': 'ឈឺខ្នង',
    'constipation': 'រំខានពោះវៀន',
    'abdominal_pain': 'ឈឺក្នុងក្រលៀន',
    'diarrhoea': 'រាក',
    'mild_fever': 'គ្រុនស្រាល',
    'yellow_urine': 'ទឹកនោមលឿង',
    'yellowing_of_eyes': 'ភ្នែកលឿង',
    'acute_liver_failure': 'ការបរាជ័យក្រពះ',
    'fluid_overload': 'ការកាត់បន្ថយហើយគ្មានអ្វីកើតឡើង',
    'swelling_of_stomach': 'ការហើមពោះ',
    'swelled_lymph_nodes': 'ការហើមកញ្ចឹងទឹក',
    'malaise': 'សេចក្តីលំបាក',
    'blurred_and_distorted_vision': 'ភាពភ្លឺខុសគ្នានិងខូចខាត',
    'phlegm': 'ការជិតឬគយ',
    'throat_irritation': 'ការលំបាកបំពង់ក',
    'redness_of_eyes': 'ភ្នែកក្រហម',
    'sinus_pressure': 'សម្ពាធនៃបំពង់មុខ',
    'runny_nose': 'ច្រមុះហូរទឹក',
    'congestion': 'ការរមៀត',
    'chest_pain': 'ឈឺទ្រូង',
    'weakness_in_limbs': 'ភាពទន់ខ្សោយក្នុងសាច់ដុំ',
    'fast_heart_rate': 'អត្រាបេះដូងលឿន',
    'pain_during_bowel_movements': 'ឈឺនៅពេលបន្តក់រោម',
    'pain_in_anal_region': 'ឈឺនៅក្នុងត្រឡាច',
    'bloody_stool': 'ចំណីឈាម',
    'irritation_in_anus': 'ការលំបាកក្នុងត្រឡាច',
    'neck_pain': 'ឈឺក',
    'dizziness': 'ឧទ្ទិស',
    'cramps': 'អន្ទាក់',
    'bruising': 'ស្នាមជាំ',
    'obesity': 'ជំងឺអាហារលើស',
    'swollen_legs': 'ជើងហើម',
    'swollen_blood_vessels': 'សរសៃឈាមហើម',
    'puffy_face_and_eyes': 'មុខនិងភ្នែកហើម',
    'enlarged_thyroid': 'ក្រពះធំ',
    'brittle_nails': 'ក្រចកខូច',
    'swollen_extremities': 'សាច់ដុំខ្សោយ',
    'excessive_hunger': 'ឃ្លាន',
    'extra_marital_contacts': 'ទំនាក់ទំនងក្រៅពីអាពាហ៍ពិពាហ៍',
    'drying_and_tingling_lips': 'សន្សើមនិងឈឺមាត់',
    'slurred_speech': 'ការនិយាយលឿន',
    'knee_pain': 'ឈឺជង្គង់',
    'hip_joint_pain': 'ឈឺសន្លាក់',
    'muscle_weakness': 'ភាពទន់ខ្សោយសាច់ដុំ',
    'stiff_neck': 'ករឹង',
    'swelling_joints': 'សន្លាក់ហើម',
    'movement_stiffness': 'ភាពរឹងក្នុងចលនា',
    'spinning_movements': 'ចលនាកោង',
    'loss_of_balance': 'បាត់សមទិន',
    'unsteadiness': 'មិនទៀងទាត់',
    'weakness_of_one_body_side': 'ទន់ខ្សោយក្នុងចំហៀងមួយ',
    'loss_of_smell': 'បាត់ចរិតក្លិន',
    'bladder_discomfort': 'ការមិនស្រួលក្នុងខ្នងប្លែត',
    'foul_smell_of_urine': 'ក្លិនស្អុយនៃទឹកនោម',
    'continuous_feel_of_urine': 'មានអារម្មណ៍បន្តបន្ទាប់នៃទឹកនោម',
    'passage_of_gases': 'ការបញ្ចេញហ្គាស',
    'internal_itching': 'ឈឺសាច់ដុំផ្នែកក្នុង',
    'toxic_look_(typhos)': 'រូបរាងពុល',
    'depression': 'ជំងឺជំងឺ',
    'irritability': 'ភាពរំខាន',
    'muscle_pain': 'ឈឺសាច់ដុំ',
    'altered_sensorium': 'ភាពផ្លាស់ប្តូរនៃចរិត',
    'red_spots_over_body': 'ចំណុចក្រហមលើខ្លួន',
    'belly_pain': 'ឈឺពោះ',
    'abnormal_menstruation': 'ប្រការជំងឺចុងសម្រាម',
    'dischromic_patches': 'ស្នាមបន្ទាត់ខុសគ្នា',
    'watering_from_eyes': 'ទឹកហូរពីភ្នែក',
    'increased_appetite': 'ពោះវៀនបន្ថែម',
    'polyuria': 'ការបញ្ចេញទឹកនោមច្រើន',
    'family_history': 'ប្រវត្តិសាច់ញាតិ',
    'mucoid_sputum': 'ទឹកនោមមានភ្លឺ',
    'rusty_sputum': 'ទឹកភ្លឺចាស់',
    'lack_of_concentration': 'បាត់បង់ការយល់ដឹង',
    'visual_disturbances': 'ការលំបាកក្នុងការមើល',
    'receiving_blood_transfusion': 'ការទទួលការបញ្ចូលឈាម',
    'receiving_unsterile_injections': 'ការទទួលការចាក់វ៉ាក់ស៊ីនមិនស្អាត',
    'coma': 'ជំងឺដេកមិនដឹងខ្លួន',
    'stomach_bleeding': 'ឈាមហូរពោះ',
    'distention_of_abdomen': 'ការហើមនៃផ្ទៃពោះ',
    'history_of_alcohol_consumption': 'ប្រវត្តិការផឹកស្រា',
    'fluid_overload': 'ការទទួលទឹកដោះ',
    'blood_in_sputum': 'ឈាមក្នុងទឹកនោម',
    'prominent_veins_on_calf': 'សរសៃឈាមបំពង់ជើងធំ',
    'palpitations': 'ការបេះដូងលឿន',
    'painful_walking': 'ឈឺក្នុងការដើរ',
    'pus_filled_pimples': 'មុនស្រស់មានទឹករាគ',
    'blackheads': 'មុនខ្មៅ',
    'scurring': 'ស្ពឹកស្បែក',
    'skin_peeling': 'ស្បែកបក',
    'silver_like_dusting': 'សម្អាតពណ៌ប្រាក់',
    'small_dents_in_nails': 'ស្នាមជាំតូចលើក្រចក',
    'inflammatory_nails': 'ការរលាកក្រចក',
    'blister': 'ពងមុន',
    'red_sore_around_nose': 'ស្នាមជាំក្រហមជុំវិញច្រមុះ',
    'yellow_crust_ooze': 'ស្នាមរបួសលឿង'
}

# Translation dictionary for prognoses (example translations, please replace with actual translations)
prognosis_translations = {
    'Fungal infection': 'ការឆ្លងប៉រ៉ាស៊ីត',
    'Allergy': 'ការពុលចំពោះអាហារឬរបស់បរិភោគ',
    'GERD': 'ជំងឺ GERD',
    'Chronic cholestasis': 'ជំងឺគ្រុន',
    'Drug Reaction': 'ប្រតិកម្មថ្នាំ',
    'Peptic ulcer diseae': 'ជំងឺផ្តាសស្រអែមក្នុងក្រពះ',
    'AIDS': 'ជំងឺអេដស៍',
    'Diabetes': 'ជំងឺទឹកនោមផ្អែម',
    'Gastroenteritis': 'ជំងឺផ្តាសពោះវៀន',
    'Bronchial Asthma': 'ជំងឺហឺតហាប់',
    'Hypertension': 'ជំងឺតឹងសរសៃឈាម',
    'Migraine': 'ឈឺក្បាលរ៉ាំរ៉ៃ',
    'Cervical spondylosis': 'រលាកកស្រាល',
    'Paralysis (brain hemorrhage)': 'សរសៃឈាមខួរក្បាលលើកជ្រួល',
    'Jaundice': 'ជំងឺលើម',
    'Malaria': 'ជំងឺគ្រុនចាញ់',
    'Chicken pox': 'ជំងឺអុំមម',
    'Dengue': 'ជំងឺគ្រុនឈាម',
    'Typhoid': 'ជំងឺថ្លោះ',
    'hepatitis A': 'ជំងឺអេរ៉ា',
    'Hepatitis B': 'ជំងឺប៊ី',
    'Hepatitis C': 'ជំងឺស៊ី',
    'Hepatitis D': 'ជំងឺឌី',
    'Hepatitis E': 'ជំងឺអ៊ី',
    'Alcoholic hepatitis': 'ជំងឺលើមដែលកើតចេញពីមេរោគ',
    'Tuberculosis': 'ជំងឺរោគ',
    'Common Cold': 'ផ្តាសាយធម្មតា',
    'Pneumonia': 'ជំងឺហឺតហាប់',
    'Dimorphic hemmorhoids(piles)': 'ជំងឺហឺតលើបន្តក់',
    'Heart attack': 'ដាច់សរសៃបេះដូង',
    'Varicose veins': 'ជំងឺសរសៃឈាមហើម',
    'Hypothyroidism': 'ជំងឺរលាកកស្រាល',
    'Hyperthyroidism': 'ជំងឺរលាកកធ្ងន់',
    'Hypoglycemia': 'ជំងឺបន្ថយស្ករឈាម',
    'Osteoarthristis': 'ជំងឺឆ្អឹង',
    'Arthritis': 'ជំងឺសន្លាក់',
    '(vertigo) Paroymsal  Positional Vertigo': 'ភាពវើហើយខ្សោយ',
    'Acne': 'មុន',
    'Urinary tract infection': 'ការប៉ះពាល់ផ្លូវទឹកនោម',
    'Psoriasis': 'ជំងឺស្បែកពន់',
    'Impetigo': 'ជំងឺផ្តាស'
}

# Create a dropdown to select the number of symptoms
num_symptoms = st.number_input('How many symptoms do you have?', min_value=1, max_value=len(feature_names), step=1)

# Create a list to hold the user's symptoms
user_symptoms = []
for i in range(num_symptoms):
    symptom = st.selectbox(f'Select symptom {i+1}', [f"{translations.get(symptom, symptom)} ({symptom})" for symptom in feature_names], key=f'symptom_{i}')
    user_symptoms.append(symptom)

# Initialize the input data with all symptoms set to 0 (No)
input_data = {feature: 0 for feature in feature_names}

# Set the input data to 1 (Yes) for the selected symptoms
for symptom in user_symptoms:
    symptom_key = symptom.split(" (")[-1][:-1]  # Extracting the English symptom name from the dropdown selection
    input_data[symptom_key] = 1

# Convert inputs to a DataFrame
input_df = pd.DataFrame([input_data])

# Button to make prediction
if st.button('Predict Disease'):
    missing_cols = set(feature_names) - set(input_df.columns)
    for col in missing_cols:
        input_df[col] = 0
    input_df = input_df[feature_names]
    
    probabilities = model.predict_proba(input_df)[0]
    predicted_prognosis_index = probabilities.argmax()
    predicted_prognosis = le.inverse_transform([predicted_prognosis_index])[0]
    translated_prognosis = prognosis_translations.get(predicted_prognosis, predicted_prognosis)
    
    # Display the predicted prognosis
    st.success(f'Predicted Disease Prognosis: {predicted_prognosis}{translated_prognosis}')
    
    # # Display the probabilities for all possible prognoses
    # st.write("Probabilities for all prognoses:")
    # # prognosis_probabilities = {prognosis}.get(prognosis, prognosis): prob for prognosis, prob in zip(le.inverse_transform(range(len(probabilities))), probabilities)
    # prognosis_probabilities = {prognosis}{prognosis_translations}.get(prognosis, prognosis): prob for prognosis, prob in zip(le.inverse_transform(range(len(probabilities))), probabilities)
    # st.write(prognosis_probabilities)

"""

with open('app.py', 'w', encoding='utf-8') as f:
    f.write(app_code)



In [5]:
# app_code = """
# import streamlit as st
# import pandas as pd
# import pickle

# # Load the trained model and label encoder
# with open('disease_model.pkl', 'rb') as f:
#     model = pickle.load(f)
# with open('label_encoder.pkl', 'rb') as f:
#     le = pickle.load(f)

# st.title('Disease Prognosis Prediction System')

# # Load and clean the feature names from the dataset
# data = pd.read_csv('diseasedata.csv')
# data.columns = data.columns.str.strip()
# data.rename(columns={
#     'dischromic _patches': 'dischromic_patches',
#     'spotting_ urination': 'spotting_urination',
#     'fluid_overload.1': 'fluid_overload'
# }, inplace=True)
# if 'Unnamed: 133' in data.columns:
#     data = data.drop(columns=['Unnamed: 133'])

# feature_names = data.columns[:-1].tolist()

# # Translation dictionary for symptoms
# translations = {
#     'itching': 'រមាស់',
#     'skin_rash': 'រលាកស្បែក',
#     'nodal_skin_eruptions': 'រោគសើរស្បែក',
#     'continuous_sneezing': 'កណ្តស់ជាប់ៗគ្នា',
#     'shivering': 'ញ័រញ៉ាក់',
#     'chills': 'រងារ',
#     'joint_pain': 'ឈឺសន្លាក់',
#     'stomach_pain': 'ឈឺពោះ',
#     'acidity': 'ឡើងជាត់អាស៊ីត',
#     'ulcers_on_tongue': 'ស្នាមជាំលើអណ្តាត',
#     'muscle_wasting': 'របួសដាច់សាច់ដុំ',
#     'vomiting': 'ហើម',
#     'burning_micturition': 'ឈឺនៅពេលបញ្ចេញទឹកនោម',
#     'spotting_urination': 'នោមញឹក',
#     'fatigue': 'អស់កំលាំង',
#     'weight_gain': 'ឡើងទម្ងន់',
#     'anxiety': 'ខ្វល់ខ្វាយច្រើន',
#     'cold_hands_and_feets': 'ដៃនិងជើងត្រជាក់',
#     'mood_swings': 'ប្តូរអារម្មណ៍គិតច្រើន',
#     'weight_loss': 'ស្រកគីឡួ',
#     'restlessness': 'ភាពមិនសុខស្រួលក្នុងខ្លួន',
#     'lethargy': 'ភាពអស់កំលាំង',
#     'patches_in_throat': 'បន្ទាត់ក្នុងបំពង់ក',
#     'irregular_sugar_level': 'កម្រិតស្ករមិនទៀងទាត់',
#     'cough': 'ក្អក',
#     'high_fever': 'គ្រុនខ្លាំង',
#     'sunken_eyes': 'ភ្នែកចុះច្រាស',
#     'breathlessness': 'ភាពមិនអាចដកដង្ហើមបាន',
#     'sweating': 'ការបែកញើស',
#     'dehydration': 'ភាពស្រេកទឹក',
#     'indigestion': 'ការមិនទទួលទានអាហារបានល្អ',
#     'headache': 'ឈឺក្បាល',
#     'yellowish_skin': 'ស្បែកលឿង',
#     'dark_urine': 'ទឹកនោមងងឹត',
#     'nausea': 'ការមិនស្រួលក្នុងពោះ',
#     'loss_of_appetite': 'បាត់បង់ចំណង់អាហារ',
#     'pain_behind_the_eyes': 'ឈឺពីក្រោយភ្នែក',
#     'back_pain': 'ឈឺខ្នង',
#     'constipation': 'រំខានពោះវៀន',
#     'abdominal_pain': 'ឈឺក្នុងក្រលៀន',
#     'diarrhoea': 'រាក',
#     'mild_fever': 'គ្រុនស្រាល',
#     'yellow_urine': 'ទឹកនោមលឿង',
#     'yellowing_of_eyes': 'ភ្នែកលឿង',
#     'acute_liver_failure': 'ការបរាជ័យក្រពះ',
#     'fluid_overload': 'ការកាត់បន្ថយហើយគ្មានអ្វីកើតឡើង',
#     'swelling_of_stomach': 'ការហើមពោះ',
#     'swelled_lymph_nodes': 'ការហើមកញ្ចឹងទឹក',
#     'malaise': 'សេចក្តីលំបាក',
#     'blurred_and_distorted_vision': 'ភាពភ្លឺខុសគ្នានិងខូចខាត',
#     'phlegm': 'ការជិតឬគយ',
#     'throat_irritation': 'ការលំបាកបំពង់ក',
#     'redness_of_eyes': 'ភ្នែកក្រហម',
#     'sinus_pressure': 'សម្ពាធនៃបំពង់មុខ',
#     'runny_nose': 'ច្រមុះហូរទឹក',
#     'congestion': 'ការរមៀត',
#     'chest_pain': 'ឈឺទ្រូង',
#     'weakness_in_limbs': 'ភាពទន់ខ្សោយក្នុងសាច់ដុំ',
#     'fast_heart_rate': 'អត្រាបេះដូងលឿន',
#     'pain_during_bowel_movements': 'ឈឺនៅពេលបន្តក់រោម',
#     'pain_in_anal_region': 'ឈឺនៅក្នុងត្រឡាច',
#     'bloody_stool': 'ចំណីឈាម',
#     'irritation_in_anus': 'ការលំបាកក្នុងត្រឡាច',
#     'neck_pain': 'ឈឺក',
#     'dizziness': 'ឧទ្ទិស',
#     'cramps': 'អន្ទាក់',
#     'bruising': 'ស្នាមជាំ',
#     'obesity': 'ជំងឺអាហារលើស',
#     'swollen_legs': 'ជើងហើម',
#     'swollen_blood_vessels': 'សរសៃឈាមហើម',
#     'puffy_face_and_eyes': 'មុខនិងភ្នែកហើម',
#     'enlarged_thyroid': 'ក្រពះធំ',
#     'brittle_nails': 'ក្រចកខូច',
#     'swollen_extremities': 'សាច់ដុំខ្សោយ',
#     'excessive_hunger': 'ឃ្លាន',
#     'extra_marital_contacts': 'ទំនាក់ទំនងក្រៅពីអាពាហ៍ពិពាហ៍',
#     'drying_and_tingling_lips': 'សន្សើមនិងឈឺមាត់',
#     'slurred_speech': 'ការនិយាយលឿន',
#     'knee_pain': 'ឈឺជង្គង់',
#     'hip_joint_pain': 'ឈឺសន្លាក់',
#     'muscle_weakness': 'ភាពទន់ខ្សោយសាច់ដុំ',
#     'stiff_neck': 'ករឹង',
#     'swelling_joints': 'សន្លាក់ហើម',
#     'movement_stiffness': 'ភាពរឹងក្នុងចលនា',
#     'spinning_movements': 'ចលនាកោង',
#     'loss_of_balance': 'បាត់សមទិន',
#     'unsteadiness': 'មិនទៀងទាត់',
#     'weakness_of_one_body_side': 'ទន់ខ្សោយក្នុងចំហៀងមួយ',
#     'loss_of_smell': 'បាត់ចរិតក្លិន',
#     'bladder_discomfort': 'ការមិនស្រួលក្នុងខ្នងប្លែត',
#     'foul_smell_of_urine': 'ក្លិនស្អុយនៃទឹកនោម',
#     'continuous_feel_of_urine': 'មានអារម្មណ៍បន្តបន្ទាប់នៃទឹកនោម',
#     'passage_of_gases': 'ការបញ្ចេញហ្គាស',
#     'internal_itching': 'ឈឺសាច់ដុំផ្នែកក្នុង',
#     'toxic_look_(typhos)': 'រូបរាងពុល',
#     'depression': 'ជំងឺជំងឺ',
#     'irritability': 'ភាពរំខាន',
#     'muscle_pain': 'ឈឺសាច់ដុំ',
#     'altered_sensorium': 'ភាពផ្លាស់ប្តូរនៃចរិត',
#     'red_spots_over_body': 'ចំណុចក្រហមលើខ្លួន',
#     'belly_pain': 'ឈឺពោះ',
#     'abnormal_menstruation': 'ប្រការជំងឺចុងសម្រាម',
#     'dischromic_patches': 'ចំណុចត្រាស់ត្រចៀក',
#     'watering_from_eyes': 'ការបាញ់ទឹកពីភ្នែក',
#     'increased_appetite': 'ឃ្លាន',
#     'polyuria': 'ការបញ្ចេញទឹកនោមច្រើន',
#     'family_history': 'ប្រវត្តិគ្រួសារ',
#     'mucoid_sputum': 'ស្លឹកឈាម',
#     'rusty_sputum': 'ស្លឹកឈាម',
#     'lack_of_concentration': 'ខ្វះការយកចិត្តទុកដាក់',
#     'visual_disturbances': 'ភាពរំខាននៃចរិត',
#     'receiving_blood_transfusion': 'ទទួលឈាមឆ្លង',
#     'receiving_unsterile_injections': 'ទទួលការចាក់ថ្នាំមិនស្អាត',
#     'coma': 'គ្មានការស្មោះសំចិត',
#     'stomach_bleeding': 'ការឈាមក្នុងក្រលៀន',
#     'distention_of_abdomen': 'ភាពហើមនៃពោះ',
#     'history_of_alcohol_consumption': 'ប្រវត្តិការផឹកស្រា',
#     'blood_in_sputum': 'ឈាមក្នុងស្លឹកឈាម',
#     'prominent_veins_on_calf': 'សរសៃឈាមធំនៅស្បែក',
#     'palpitations': 'ការបេះដូងលឿន',
#     'painful_walking': 'ការដើរឈឺ',
#     'pus_filled_pimples': 'បំពង់ចង្កេះ',
#     'blackheads': 'ច្រមុះ',
#     'scurring': 'ការខ្វះនៃសាច់ដុំ',
#     'skin_peeling': 'ស្បែកកោស',
#     'silver_like_dusting': 'ការបន្សាប្រមូលធូលីដូចប្រាក់',
#     'small_dents_in_nails': 'បំបែកក្នុងក្រចក',
#     'inflammatory_nails': 'ក្រចកកើតឡើង',
#     'blister': 'ស្លឹកឈាម',
#     'red_sore_around_nose': 'ស្នាមជាំក្រហមនៅច្រមុះ',
#     'yellow_crust_ooze': 'ការឈាមលឿង'
# }
# # Create checkboxes for each symptom
# selected_symptoms = []
# for symptom in feature_names:
#     if st.checkbox(f'{translations.get(symptom, symptom)} ({symptom})'):
#         selected_symptoms.append(symptom)

# # Initialize the input data with all symptoms set to 0 (No)
# input_data = {feature: 0 for feature in feature_names}

# # Set the input data to 1 (Yes) for the selected symptoms
# for symptom in selected_symptoms:
#     input_data[symptom] = 1

# # Convert inputs to a DataFrame
# input_df = pd.DataFrame([input_data])

# # Button to make prediction
# if st.button('Predict Disease'):
#     missing_cols = set(feature_names) - set(input_df.columns)
#     for col in missing_cols:
#         input_df[col] = 0
#     input_df = input_df[feature_names]
    
#     probabilities = model.predict_proba(input_df)[0]
#     predicted_prognosis_index = probabilities.argmax()
#     predicted_prognosis = le.inverse_transform([predicted_prognosis_index])[0]
#     translated_prognosis = prognosis_translations.get(predicted_prognosis, predicted_prognosis)
    
#     # Display the predicted prognosis
#     st.success(f'Predicted Disease Prognosis: {prognosis}')
# """
# with open('app.py', 'w', encoding='utf-8') as f:
#   f.write(app_code)


In [6]:
import subprocess
subprocess.run(['streamlit', 'run', 'app.py'])
